In [ ]:
# Тестируем модель на тестовом наборе данных
total_loss = 0
total_psnr = 0
total_samples = 0

with torch.no_grad():  # Отключаем градиенты для ускорения
    for L, ab in test_loader:
        L, ab = L.to(device), ab.to(device)

        # Прогоняем через модель
        ab_pred = model(L)

        # Считаем ошибку
        loss = criterion(ab_pred, ab)
        total_loss += loss.item() * L.size(0)  # Умножаем на размер батча
        total_samples += L.size(0)

        # Рассчитываем PSNR
        psnr_value = psnr(ab_pred, ab)
        total_psnr += psnr_value.item()

# Средние значения метрик
avg_test_loss = total_loss / total_samples
avg_psnr = total_psnr / total_samples

# Печатаем результаты
print(f"Средняя ошибка на тестовом наборе (MSE): {avg_test_loss:.4f}")
print(f"Средний PSNR: {avg_psnr:.4f} dB")


In [ ]:
model.eval()

L, ab = test_dataset[0]

L_input = L.unsqueeze(0).to(device)

with torch.no_grad():
  ab_pred = model(L_input)

ab_pred = ab_pred.squeeze(0).cpu()
L_np = L.squeeze().numpy()*100
ab_np = ab_pred.numpy().transpose(1,2,0)*128

lab_pred = np.concatenate([L_np[..., np.newaxis], ab_np], axis=2)  # (32, 32, 3)
rgb_pred = color.lab2rgb(lab_pred)

original_rgb = test_dataset.data.data[0]  # uint8 RGB
original_rgb = original_rgb  # нормализуем [0, 1]

# Визуализация
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

axs[0].imshow(original_rgb)
axs[0].set_title("Оригинал")
axs[0].axis("off")

axs[1].imshow(rgb_pred)
axs[1].set_title("Предсказание")
axs[1].axis("off")